In [26]:
import bittensor as bt
import random
import os
import sys
import numpy as np
import pandas as pd
import hashlib
from collections import defaultdict
from Crypto import Random
from Crypto.Random import random
from Crypto.PublicKey import ECC
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
from pprint import pprint, pformat
import binascii


In [27]:
class ECCommitment:
    def __init__(self, g, h):
        self.g = g  # Base point of the curve
        self.h = h  # Another random point on the curve

    def commit(self, m):  # AKA Seal.
        m_val = hash_data(m)  # Compute hash of the data
        r = random.randint(1, 2**256)
        c1 = self.g.__mul__(m_val)
        c2 = self.h.__mul__(r)
        c = c1.__add__(c2)
        print(
            f"Committing: Data = {m}\nHashed Value = {m_val}\nRandom Value = {r}\nComputed Commitment = {c}\n"
        )
        return c, m_val, r

    def open(self, c, m_val, r):
        c1 = self.g.__mul__(m_val)
        c2 = self.h.__mul__(r)
        computed_c = c1.__add__(c2)
        print(
            f"\nOpening: Hashed Value = {m_val}\nRandom Value = {r}\nRecomputed Commitment = {computed_c}\nOriginal Commitment = {c}"
        )
        return computed_c == c


def setup_CRS():
    curve = ECC.generate(curve="P-256")
    g = curve.pointQ  # Base point
    h = ECC.generate(curve="P-256").pointQ  # Another random point
    return g, h


In [28]:
g, h = setup_CRS()
print("g:", g)
print("h:", h)

g: <Crypto.PublicKey.ECC.EccPoint object at 0x7ff556fa1d80>
h: <Crypto.PublicKey.ECC.EccPoint object at 0x7ff556fa33a0>


In [29]:
def make_random_file(name=None, maxsize=1024):
    size = random.randint(32, maxsize)
    data = os.urandom(size)
    if isinstance(name, str):
        with open(name, "wb") as fout:
            fout.write(data)
    else:
        return data

f = random_file = make_random_file()
len(f)

369

In [30]:
def encrypt_data(filename, key):
    """
    Encrypt the data in the given filename using AES-GCM.

    Parameters:
    - filename: str or bytes. If str, it's considered as a file name. If bytes, as the data itself.
    - key: bytes. 16-byte (128-bit), 24-byte (192-bit), or 32-byte (256-bit) secret key.

    Returns:
    - cipher_text: bytes. The encrypted data.
    - nonce: bytes. The nonce used for the GCM mode.
    - tag: bytes. The tag for authentication.
    """

    # If filename is a string, treat it as a file name and read the data
    if isinstance(filename, str):
        with open(filename, "rb") as file:
            data = file.read()
    else:
        data = filename

    # Initialize AES-GCM cipher
    cipher = AES.new(key, AES.MODE_GCM)
    
    # Encrypt the data
    cipher_text, tag = cipher.encrypt_and_digest(data)

    return cipher_text, cipher.nonce, tag

# Example usage
key = get_random_bytes(32)  # 256-bit key
encrypted_data, nonce, tag = encrypt_data(random_file, key)
encrypted_data

b'\x8c\x00\x9c[\xd3\x84\xe9\xd9\xf6\x1a\x98\xb7\xcb\xc9\xb9\x0c9\x03L\xad\x1f\xd3\xba[\x96\x17\xb1\xa3\x93\xde\xd1\xf2\xfdy\xb3\xd5a(\xdd\xeb3D\xc5C\xfcLc\xd8d\x9d\x86O-A\x19\xbe\xe6On\x83\x1cUT\xb4\x12\x8c\xfdQ{\x98\xec3\xd2\xbe\xd9\xc4\xcc]\xff{,\x00\x80\'\x87\x86b \x14\xc6ZAl\xceC\xc6\x9d\x81\xfa\xd6\x1fH\x0f#O\x84\x93\xb1\xd4\x87\xe8\x85a\xfd\x99-\xdd\xeak\xa0\xaf\x12\xd2A\x15\\\x92*\x99\xb2>\xb4\x14\x14C\x122\xe0\xe6\xf6s\x8f\xc4K\x92:\xf8f\x92\x87\xfd\x8eM\xfdd"z\xbb\xb9-\x9b\x15\xa1\xee\x12\x1d\xc4i 5\xe8\x1a\x9b/\xd4\xaeC+&\x85\xcf"!(SMB\xca=\xc1\xa4h\xdak$)\xf2-#\xb9;\x00\xff\xd6I\xa4\xa5\xc8\xf9\x9e\x87kx\xb1\xa2\xdf\x93\xcf\xf4\x8f5BP\xb5\x1cc\x11\x88!%\x0cs\xabp2\n,;\xccu\xe3\xf0\xb6\xea\xa0F02t\x92t\x94\xa7kj\x8f\xe0\x83\xc3\x88\\5\x8e\x89J\xec\x0b;})6\xa1W"R\x95#2\xab\xd8\x01\xdf\xe8X\xb3!\t\xff\xec"\xaam\x18\x00\x87f\xac9\x907\xab-\xb3\x1a\x15E\xfb\x82\xd8L\'\x16\xda\xb8\xb1f\xe7lMw\x01\xbb\x00S\x83\xeb\xa0\x8c\xff3=\xf5\xd5A\x17\xc78\x11\xdb\x90=f\x0bC1\xec6\x0b<\xa4\x9

In [31]:
# Hash encrypted data E with SHA3-256 (args: encrypted data E) to get hash H
def hash_data(data):
    if not isinstance(data, (bytes, bytearray)):
        data_str = str(data)
        data = data_str.encode()
    h = hashlib.sha3_256(data).hexdigest()
    return int(h, 16)

H = hash_data(encrypted_data)
H

103840597209884809975283021825466613051704358558676079751266779814020477816203

In [32]:
# Determine a random chunksize between 2kb-128kb (random sample from this range) store as chunksize_E
def get_random_chunksize(maxsize=128):
    return random.randint(2, maxsize)

bytesize = sys.getsizeof(encrypted_data)
print("bytesize:", bytesize)
chunksize = get_random_chunksize(bytesize // 4)
print("chunksize:", chunksize)

bytesize: 402
chunksize: 32


In [33]:
def chunk_data(data, chunksize: int):
    for i in range(0, len(data), chunksize):
        yield data[i : i + chunksize]

chunks = list(chunk_data(encrypted_data, chunksize))
print("n chunks:", len(chunks))
pprint(f"chunks: {chunks}")

n chunks: 12
('chunks: '
 "[b'\\x8c\\x00\\x9c[\\xd3\\x84\\xe9\\xd9\\xf6\\x1a\\x98\\xb7\\xcb\\xc9\\xb9\\x0c9\\x03L\\xad\\x1f\\xd3\\xba[\\x96\\x17\\xb1\\xa3\\x93\\xde\\xd1\\xf2', "
 "b'\\xfdy\\xb3\\xd5a(\\xdd\\xeb3D\\xc5C\\xfcLc\\xd8d\\x9d\\x86O-A\\x19\\xbe\\xe6On\\x83\\x1cUT\\xb4', "
 'b"\\x12\\x8c\\xfdQ{\\x98\\xec3\\xd2\\xbe\\xd9\\xc4\\xcc]\\xff{,\\x00\\x80\'\\x87\\x86b '
 '\\x14\\xc6ZAl\\xceC\\xc6", '
 "b'\\x9d\\x81\\xfa\\xd6\\x1fH\\x0f#O\\x84\\x93\\xb1\\xd4\\x87\\xe8\\x85a\\xfd\\x99-\\xdd\\xeak\\xa0\\xaf\\x12\\xd2A\\x15\\\\\\x92*', "
 'b\'\\x99\\xb2>\\xb4\\x14\\x14C\\x122\\xe0\\xe6\\xf6s\\x8f\\xc4K\\x92:\\xf8f\\x92\\x87\\xfd\\x8eM\\xfdd"z\\xbb\\xb9-\', '
 "b'\\x9b\\x15\\xa1\\xee\\x12\\x1d\\xc4i "
 '5\\xe8\\x1a\\x9b/\\xd4\\xaeC+&\\x85\\xcf"!(SMB\\xca=\\xc1\\xa4h\', '
 "b'\\xdak$)\\xf2-#\\xb9;\\x00\\xff\\xd6I\\xa4\\xa5\\xc8\\xf9\\x9e\\x87kx\\xb1\\xa2\\xdf\\x93\\xcf\\xf4\\x8f5BP\\xb5', "
 "b'\\x1cc\\x11\\x88!%\\x0cs\\xabp2\\n,;\\xccu\\xe3\\xf0\\xb6\\xea\\xa0F02t\\x92t\\x94\\xa7kj\\x8f',

In [34]:
committer = ECCommitment(g, h)
committer

In [35]:
class MerkleTree(object):
    def __init__(self, hash_type="sha3_256"):
        hash_type = hash_type.lower()
        if hash_type in [
            "sha256",
            "sha224",
            "sha384",
            "sha512",
            "sha3_256",
            "sha3_224",
            "sha3_384",
            "sha3_512",
        ]:
            self.hash_function = getattr(hashlib, hash_type)
        else:
            raise Exception("`hash_type` {} nor supported".format(hash_type))

        self.reset_tree()

    def _to_hex(self, x):
        try:  # python3
            return x.hex()
        except:  # python2
            return binascii.hexlify(x)

    def reset_tree(self):
        self.leaves = list()
        self.levels = None
        self.is_ready = False

    def add_leaf(self, values, do_hash=False):
        self.is_ready = False
        # check if single leaf
        if not isinstance(values, tuple) and not isinstance(values, list):
            values = [values]
        for v in values:
            if do_hash:
                v = v.encode("utf-8")
                v = self.hash_function(v).hexdigest()
            v = bytearray.fromhex(v)
            self.leaves.append(v)

    def get_leaf(self, index):
        return self._to_hex(self.leaves[index])

    def get_leaf_count(self):
        return len(self.leaves)

    def get_tree_ready_state(self):
        return self.is_ready

    def _calculate_next_level(self):
        solo_leave = None
        N = len(self.levels[0])  # number of leaves on the level
        if N % 2 == 1:  # if odd number of leaves on the level
            solo_leave = self.levels[0][-1]
            N -= 1

        new_level = []
        for l, r in zip(self.levels[0][0:N:2], self.levels[0][1:N:2]):
            new_level.append(self.hash_function(l + r).digest())
        if solo_leave is not None:
            new_level.append(solo_leave)
        self.levels = [
            new_level,
        ] + self.levels  # prepend new level

    def make_tree(self):
        self.is_ready = False
        if self.get_leaf_count() > 0:
            self.levels = [
                self.leaves,
            ]
            while len(self.levels[0]) > 1:
                self._calculate_next_level()
        self.is_ready = True

    def get_merkle_root(self):
        if self.is_ready:
            if self.levels is not None:
                return self._to_hex(self.levels[0][0])
            else:
                return None
        else:
            return None

    def get_proof(self, index):
        if self.levels is None:
            return None
        elif not self.is_ready or index > len(self.leaves) - 1 or index < 0:
            return None
        else:
            proof = []
            for x in range(len(self.levels) - 1, 0, -1):
                level_len = len(self.levels[x])
                if (index == level_len - 1) and (
                    level_len % 2 == 1
                ):  # skip if this is an odd end node
                    index = int(index / 2.0)
                    continue
                is_right_node = index % 2
                sibling_index = index - 1 if is_right_node else index + 1
                sibling_pos = "left" if is_right_node else "right"
                sibling_value = self._to_hex(self.levels[x][sibling_index])
                proof.append({sibling_pos: sibling_value})
                index = int(index / 2.0)
            return proof

def validate_merkle_proof(proof, target_hash, merkle_root):
    merkle_root = bytearray.fromhex(merkle_root)
    target_hash = bytearray.fromhex(target_hash)
    if len(proof) == 0:
        return target_hash == merkle_root
    else:
        proof_hash = target_hash
        for p in proof:
            try:
                # the sibling is a left node
                sibling = bytearray.fromhex(p["left"])
                proof_hash = hashlib.sha3_256(sibling + proof_hash).digest()
            except:
                # the sibling is a right node
                sibling = bytearray.fromhex(p["right"])
                proof_hash = hashlib.sha3_256(proof_hash + sibling).digest()
        return proof_hash == merkle_root


In [36]:
def commit_data(committer, data_chunks):
    merkle_tree = MerkleTree()
    commitments = defaultdict(lambda: [None] * len(data_chunks))
    # commitments = {}

    for index, chunk in enumerate(data_chunks):
        c, m_val, r = committer.commit(chunk)
        commitments[index] = {
            "index": index,
            "hash": m_val,
            "data_chunk": chunk,
            "point": c,
            "randomness": r,
            "merkle_proof": None,
        }
        merkle_tree.add_leaf(ecc_point_to_hex(c))

    merkle_tree.make_tree()
    return {merkle_tree.get_merkle_root(): {'commitments': commitments, 'merkle_tree': merkle_tree}}

def ecc_point_to_hex(point):
    point_str = "{},{}".format(point.x, point.y)
    return binascii.hexlify(point_str.encode()).decode()

def hex_to_ecc_point(hex_str, curve):
    point_str = binascii.unhexlify(hex_str).decode()
    x, y = map(int, point_str.split(","))
    return ECC.EccPoint(curve, x, y)

store_data = commit_data
commitments_E = store_data(committer, chunks)
commitments_E
# TODO: once this is generaetd, we send it back to the Validator minus the random values
# {merkle_root: (data_chunks, hashes, commitment_points)} 

Committing: Data = b'\x8c\x00\x9c[\xd3\x84\xe9\xd9\xf6\x1a\x98\xb7\xcb\xc9\xb9\x0c9\x03L\xad\x1f\xd3\xba[\x96\x17\xb1\xa3\x93\xde\xd1\xf2'
Hashed Value = 50366001327247683482752636260973803818453526684612359972737906020344372234305
Random Value = 75597626110924612833046533695986495499515762785055351061194772398784596778329
Computed Commitment = <Crypto.PublicKey.ECC.EccPoint object at 0x7ff5572c2950>

Committing: Data = b'\xfdy\xb3\xd5a(\xdd\xeb3D\xc5C\xfcLc\xd8d\x9d\x86O-A\x19\xbe\xe6On\x83\x1cUT\xb4'
Hashed Value = 67868018335286132395530187885642597945614912859475789356043632607036172284368
Random Value = 85653675785040475053686274542074563054910376916294442745559007765224783262889
Computed Commitment = <Crypto.PublicKey.ECC.EccPoint object at 0x7ff5572c2a70>

Committing: Data = b"\x12\x8c\xfdQ{\x98\xec3\xd2\xbe\xd9\xc4\xcc]\xff{,\x00\x80'\x87\x86b \x14\xc6ZAl\xceC\xc6"
Hashed Value = 36387826893975387600457443977955486113664700015506317451365783069115925746071
Random Value = 900801

{'755ea86c12fb83d59c657f183db03820b3ca2ba9c13a06f6f4ec84054cd88b8f': {'commitments': defaultdict(<function __main__.commit_data.<locals>.<lambda>()>,
              {0: {'index': 0,
                'hash': 50366001327247683482752636260973803818453526684612359972737906020344372234305,
                'data_chunk': b'\x8c\x00\x9c[\xd3\x84\xe9\xd9\xf6\x1a\x98\xb7\xcb\xc9\xb9\x0c9\x03L\xad\x1f\xd3\xba[\x96\x17\xb1\xa3\x93\xde\xd1\xf2',
                'point': <Crypto.PublicKey.ECC.EccPoint at 0x7ff5572c2950>,
                'randomness': 75597626110924612833046533695986495499515762785055351061194772398784596778329,
                'merkle_proof': None},
               1: {'index': 1,
                'hash': 67868018335286132395530187885642597945614912859475789356043632607036172284368,
                'data_chunk': b'\xfdy\xb3\xd5a(\xdd\xeb3D\xc5C\xfcLc\xd8d\x9d\x86O-A\x19\xbe\xe6On\x83\x1cUT\xb4',
                'point': <Crypto.PublicKey.ECC.EccPoint at 0x7ff5572c2a70>,
                

In [37]:
keys = list(commitments_E.keys())
keys

['755ea86c12fb83d59c657f183db03820b3ca2ba9c13a06f6f4ec84054cd88b8f']

In [38]:
inner_dict = commitments_E[keys[0]]
inner_dict

{'commitments': defaultdict(<function __main__.commit_data.<locals>.<lambda>()>,
             {0: {'index': 0,
               'hash': 50366001327247683482752636260973803818453526684612359972737906020344372234305,
               'data_chunk': b'\x8c\x00\x9c[\xd3\x84\xe9\xd9\xf6\x1a\x98\xb7\xcb\xc9\xb9\x0c9\x03L\xad\x1f\xd3\xba[\x96\x17\xb1\xa3\x93\xde\xd1\xf2',
               'point': <Crypto.PublicKey.ECC.EccPoint at 0x7ff5572c2950>,
               'randomness': 75597626110924612833046533695986495499515762785055351061194772398784596778329,
               'merkle_proof': None},
              1: {'index': 1,
               'hash': 67868018335286132395530187885642597945614912859475789356043632607036172284368,
               'data_chunk': b'\xfdy\xb3\xd5a(\xdd\xeb3D\xc5C\xfcLc\xd8d\x9d\x86O-A\x19\xbe\xe6On\x83\x1cUT\xb4',
               'point': <Crypto.PublicKey.ECC.EccPoint at 0x7ff5572c2a70>,
               'randomness': 856536757850404750536862745420745630549103769162944427455590077652

In [39]:
hashes = inner_dict["commitments"]["hashes"]
assert len(hashes) == len(chunks)

In [40]:
def get_challenge_indices(num_chunks, factor=0.1):
    return random.sample(list(range(num_chunks)), int(1 + num_chunks * factor))

ci = challenge_indices = get_challenge_indices(len(chunks))
ci

[8, 3]

In [41]:
def get_merkle_root_to_challenge(commitments):
    merkle_roots = list(commitments.keys())
    return random.choice(merkle_roots)

mrc = merkle_root_challenge = get_merkle_root_to_challenge(commitments_E)
mrc

'755ea86c12fb83d59c657f183db03820b3ca2ba9c13a06f6f4ec84054cd88b8f'

In [42]:
responses = []
challenge_data = commitments_E[merkle_root_challenge]
merkle_tree = challenge_data['merkle_tree']
for i in challenge_indices:
    challenge_data['commitments'][i]['merkle_proof'] = merkle_tree.get_proof(i)
    responses.append(challenge_data['commitments'][i])

challenge_response = {merkle_root_challenge: responses}
challenge_response

{'755ea86c12fb83d59c657f183db03820b3ca2ba9c13a06f6f4ec84054cd88b8f': [{'index': 8,
   'hash': 61679568111664353655801833811917086127997832868533498827956442851753482737069,
   'data_chunk': b'\xe0\x83\xc3\x88\\5\x8e\x89J\xec\x0b;})6\xa1W"R\x95#2\xab\xd8\x01\xdf\xe8X\xb3!\t\xff',
   'point': <Crypto.PublicKey.ECC.EccPoint at 0x7ff5572c2ce0>,
   'randomness': 92215692208006191999454828463427100803951244580593173559674579170371219433284,
   'merkle_proof': [{'right': '343135303434363739343031333731343634353930363733393933333437363233343033333739313538303734333636383639303138313138343834333034303035303238343639303237372c313039363137393438323930343938363033393032333931373930393638363732333030303137313239363636303434393831383031363434373733353934353534343436333331383133323239'},
    {'right': 'd90542e242422c550f6ecc313c9994041d69a05e3142a4e4e3735a0657f34e12'},
    {'left': '275fd24af88283dbc39417f9eea613cb14857e088edf1636732e9572e6e8aa97'}]},
  {'index': 3,
   'hash': 21451205755144499869330

In [43]:
merkle_root = list(challenge_response.keys())[0]
commitments = challenge_response[merkle_root]
print("merkle_root:", merkle_root)
commitments

merkle_root: 755ea86c12fb83d59c657f183db03820b3ca2ba9c13a06f6f4ec84054cd88b8f


[{'index': 8,
  'hash': 61679568111664353655801833811917086127997832868533498827956442851753482737069,
  'data_chunk': b'\xe0\x83\xc3\x88\\5\x8e\x89J\xec\x0b;})6\xa1W"R\x95#2\xab\xd8\x01\xdf\xe8X\xb3!\t\xff',
  'point': <Crypto.PublicKey.ECC.EccPoint at 0x7ff5572c2ce0>,
  'randomness': 92215692208006191999454828463427100803951244580593173559674579170371219433284,
  'merkle_proof': [{'right': '343135303434363739343031333731343634353930363733393933333437363233343033333739313538303734333636383639303138313138343834333034303035303238343639303237372c313039363137393438323930343938363033393032333931373930393638363732333030303137313239363636303434393831383031363434373733353934353534343436333331383133323239'},
   {'right': 'd90542e242422c550f6ecc313c9994041d69a05e3142a4e4e3735a0657f34e12'},
   {'left': '275fd24af88283dbc39417f9eea613cb14857e088edf1636732e9572e6e8aa97'}]},
 {'index': 3,
  'hash': 21451205755144499869330940260758031991897023959950131656171117591665016862696,
  'data_chunk': b'\x9d

In [44]:
len(commitments)

2

In [45]:
commitment_i = commitments[0]
commitment_i

{'index': 8,
 'hash': 61679568111664353655801833811917086127997832868533498827956442851753482737069,
 'data_chunk': b'\xe0\x83\xc3\x88\\5\x8e\x89J\xec\x0b;})6\xa1W"R\x95#2\xab\xd8\x01\xdf\xe8X\xb3!\t\xff',
 'point': <Crypto.PublicKey.ECC.EccPoint at 0x7ff5572c2ce0>,
 'randomness': 92215692208006191999454828463427100803951244580593173559674579170371219433284,
 'merkle_proof': [{'right': '343135303434363739343031333731343634353930363733393933333437363233343033333739313538303734333636383639303138313138343834333034303035303238343639303237372c313039363137393438323930343938363033393032333931373930393638363732333030303137313239363636303434393831383031363434373733353934353534343436333331383133323239'},
  {'right': 'd90542e242422c550f6ecc313c9994041d69a05e3142a4e4e3735a0657f34e12'},
  {'left': '275fd24af88283dbc39417f9eea613cb14857e088edf1636732e9572e6e8aa97'}]}

In [46]:
merkle_proof_i = commitments[0]['merkle_proof']
merkle_proof_i

[{'right': '343135303434363739343031333731343634353930363733393933333437363233343033333739313538303734333636383639303138313138343834333034303035303238343639303237372c313039363137393438323930343938363033393032333931373930393638363732333030303137313239363636303434393831383031363434373733353934353534343436333331383133323239'},
 {'right': 'd90542e242422c550f6ecc313c9994041d69a05e3142a4e4e3735a0657f34e12'},
 {'left': '275fd24af88283dbc39417f9eea613cb14857e088edf1636732e9572e6e8aa97'}]

In [47]:
commitments[0]

{'index': 8,
 'hash': 61679568111664353655801833811917086127997832868533498827956442851753482737069,
 'data_chunk': b'\xe0\x83\xc3\x88\\5\x8e\x89J\xec\x0b;})6\xa1W"R\x95#2\xab\xd8\x01\xdf\xe8X\xb3!\t\xff',
 'point': <Crypto.PublicKey.ECC.EccPoint at 0x7ff5572c2ce0>,
 'randomness': 92215692208006191999454828463427100803951244580593173559674579170371219433284,
 'merkle_proof': [{'right': '343135303434363739343031333731343634353930363733393933333437363233343033333739313538303734333636383639303138313138343834333034303035303238343639303237372c313039363137393438323930343938363033393032333931373930393638363732333030303137313239363636303434393831383031363434373733353934353534343436333331383133323239'},
  {'right': 'd90542e242422c550f6ecc313c9994041d69a05e3142a4e4e3735a0657f34e12'},
  {'left': '275fd24af88283dbc39417f9eea613cb14857e088edf1636732e9572e6e8aa97'}]}

In [48]:
for commitment_i in commitments:
    index = commitment_i['index']
    commitment = commitment_i['point']
    data = commitment_i['data_chunk']
    r = commitment_i['randomness']
    merkle_proof_i = commitment_i['merkle_proof']

    if not committer.open(commitment, hash_data(data), r):
        print(f"Opening commitment {index} failed")

    if not validate_merkle_proof(
        merkle_proof_i, ecc_point_to_hex(commitment_i['point']), merkle_root
    ):
        print(f"Merkle proof {index} validation failed")


Opening: Hashed Value = 61679568111664353655801833811917086127997832868533498827956442851753482737069
Random Value = 92215692208006191999454828463427100803951244580593173559674579170371219433284
Recomputed Commitment = <Crypto.PublicKey.ECC.EccPoint object at 0x7ff5572c3850>
Original Commitment = <Crypto.PublicKey.ECC.EccPoint object at 0x7ff5572c2ce0>

Opening: Hashed Value = 21451205755144499869330940260758031991897023959950131656171117591665016862696
Random Value = 113094821845468040731307364861305094956015392868289188667174893976136171877464
Recomputed Commitment = <Crypto.PublicKey.ECC.EccPoint object at 0x7ff5572c3430>
Original Commitment = <Crypto.PublicKey.ECC.EccPoint object at 0x7ff5572c36d0>


In [49]:
def verify(committer, commitments, merkle_root):
    for commitment_i in commitments:
        index = commitment_i['index']
        commitment = commitment_i['point']
        data = commitment_i['data_chunk']
        r = commitment_i['randomness']
        merkle_proof_i = commitment_i['merkle_proof']

        if not committer.open(commitment, hash_data(data), r):
            print(f"Opening commitment {index} failed")
            return False

        if not validate_merkle_proof(
            merkle_proof_i, ecc_point_to_hex(commitment_i['point']), merkle_root
        ):
            print(f"Merkle proof {index} validation failed")
            return False

    return True

In [50]:
verify(committer, commitments, merkle_root)


Opening: Hashed Value = 61679568111664353655801833811917086127997832868533498827956442851753482737069
Random Value = 92215692208006191999454828463427100803951244580593173559674579170371219433284
Recomputed Commitment = <Crypto.PublicKey.ECC.EccPoint object at 0x7ff5572c1e70>
Original Commitment = <Crypto.PublicKey.ECC.EccPoint object at 0x7ff5572c2ce0>

Opening: Hashed Value = 21451205755144499869330940260758031991897023959950131656171117591665016862696
Random Value = 113094821845468040731307364861305094956015392868289188667174893976136171877464
Recomputed Commitment = <Crypto.PublicKey.ECC.EccPoint object at 0x7ff5572c38e0>
Original Commitment = <Crypto.PublicKey.ECC.EccPoint object at 0x7ff5572c36d0>


True